In [3]:
suppressPackageStartupMessages({
    library(Rsamtools)
    library(GenomicAlignments)
    library(GenomicFiles)
    library(data.table)
    library(rtracklayer)
    library(GenomicRanges)
    library(RColorBrewer)
    library(doParallel)
});

In [4]:
cfilters = ScanBamParam(mapqFilter=10, flag=scanBamFlag(isSecondaryAlignment = F, isDuplicate=F));
bfilters = ScanBamParam(mapqFilter=10, flag=scanBamFlag(isSecondaryAlignment = F));
yield.bam = function(X) {
    y = GRanges( readGAlignmentPairs(X, use.names=F, param=bfilters ));
    return(y);
}

yield.bam1 = function(X) {
    y = GRanges( readGAlignmentPairs(X, use.names=F, param=cfilters ));
    return(y);
}

map.bam = function(X) {
    return(X);
}

reduce.bam = function(x, y) {
    x = append(x, y);
    # print the number of readpairs processed
    #msgout(pn(length(x)), 'mapped human reads');
    return(x);
}

collapse_reads = function(reads) {
    print(length(reads))
    uniq = unique(reads)
    # sum over duplicates to get a count for each unique 5'/3' end
    uniq$count = countOverlaps( uniq, reads, type="equal");
    print(length(uniq))
    return( uniq );
}

In [58]:
test_500='/data/reddylab/Revathy/dev/Jamborees/MPRA_STARRseq/tmp/k562.atacstarr.output.rep3.dups_marked_subset500.bam' 
test_bam=BamFile(test_500)

## yield.bam doesn't use isDuplicate=F flag while yield.bam1 uses the isDuplicate=F flag

test_out=reduceByYield(test_bam, yield.bam, map.bam, reduce.bam)
test_out_isdup=reduceByYield(test_bam, yield.bam1, map.bam, reduce.bam)

ctTable=collapse_reads(test_out)
ctTable_isdup=collapse_reads(test_out_isdup)

[1] 249
[1] 125
[1] 128
[1] 124


In [61]:
head(ctTable_isdup, n=10)

GRanges object with 10 ranges and 1 metadata column:
       seqnames         ranges strand |     count
          <Rle>      <IRanges>  <Rle> | <integer>
   [1]     chr1 [10586, 11175]      - |         1
   [2]     chr1 [13149, 13328]      + |         1
   [3]     chr1 [13170, 13341]      + |         1
   [4]     chr1 [13304, 13494]      + |         1
   [5]     chr1 [13304, 13505]      + |         1
   [6]     chr1 [13304, 13505]      - |         2
   [7]     chr1 [13315, 13505]      - |         1
   [8]     chr1 [14453, 14620]      + |         1
   [9]     chr1 [14694, 15109]      + |         1
  [10]     chr1 [15180, 15401]      + |         2
  -------
  seqinfo: 195 sequences from an unspecified genome

In [62]:
ctTable_df = as.data.frame(ctTable_isdup)
newCols=data.frame(name=paste0(ctTable_df$seqnames,"_", ctTable_df$start,"_",ctTable_df$end),
                   score = pmin(ctTable_df$count,1000),
                   barcode = '.')
newCols

# ctTable_df$name = paste0(ctTable_df$seqnames,"_", ctTable_df$start,"_",ctTable_df$end)
# ctTable_df$score = '1000'
# ctTable_df$barcode = '.'
# col_order = c('seqnames','start','end','name','score','strand','count','barcode')
# ctTable_df = ctTable_df[, col_order]
# ctTable_df

name,score,barcode
chr1_10586_11175,1,.
chr1_13149_13328,1,.
chr1_13170_13341,1,.
chr1_13304_13494,1,.
chr1_13304_13505,1,.
chr1_13304_13505,2,.
chr1_13315_13505,1,.
chr1_14453_14620,1,.
chr1_14694_15109,1,.
chr1_15180_15401,2,.


In [39]:
write.csv(test_out_isdup, "/data/reddylab/Revathy/dev/Jamborees/MPRA_STARRseq/tmp/test_out_isdup_table.txt")

In [35]:
head(test_out_isdup, n=10)

GRanges object with 10 ranges and 0 metadata columns:
       seqnames         ranges strand
          <Rle>      <IRanges>  <Rle>
   [1]     chr1 [10586, 11175]      -
   [2]     chr1 [13149, 13328]      +
   [3]     chr1 [13170, 13341]      +
   [4]     chr1 [13304, 13494]      +
   [5]     chr1 [13304, 13505]      +
   [6]     chr1 [13304, 13505]      -
   [7]     chr1 [13315, 13505]      -
   [8]     chr1 [13304, 13505]      -
   [9]     chr1 [14453, 14620]      +
  [10]     chr1 [14694, 15109]      +
  -------
  seqinfo: 195 sequences from an unspecified genome

In [28]:
test_1000='/data/reddylab/Revathy/dev/Jamborees/MPRA_STARRseq/tmp/k562.atacstarr.output.rep3.dups_marked_subset1000.bam'
test_bam=BamFile(test_1000)

## yield.bam doesn't use isDuplicate=F flag while yield.bam1 uses the isDuplicate=F flag

test_out=reduceByYield(test_bam, yield.bam, map.bam, reduce.bam)
test_out_isdup=reduceByYield(test_bam, yield.bam1, map.bam, reduce.bam)

ctTable=collapse_reads(test_out)
ctTable_isdup=collapse_reads(test_out_isdup)

[1] 465
[1] 223
[1] 242
[1] 221


In [26]:
ctTable

GRanges object with 223 ranges and 1 metadata column:
        seqnames           ranges strand |     count
           <Rle>        <IRanges>  <Rle> | <integer>
    [1]     chr1   [10586, 11175]      - |         2
    [2]     chr1   [13149, 13328]      + |         1
    [3]     chr1   [13170, 13341]      + |         3
    [4]     chr1   [13304, 13494]      + |         3
    [5]     chr1   [13304, 13505]      + |         2
    ...      ...              ...    ... .       ...
  [219]     chr1 [115638, 115758]      - |         1
  [220]     chr1 [115637, 115758]      - |         1
  [221]     chr1 [115676, 115758]      - |         2
  [222]     chr1 [115650, 115758]      - |         1
  [223]     chr1 [115558, 115758]      - |         1
  -------
  seqinfo: 195 sequences from an unspecified genome

In [25]:
ctTable_isdup

GRanges object with 221 ranges and 1 metadata column:
        seqnames           ranges strand |     count
           <Rle>        <IRanges>  <Rle> | <integer>
    [1]     chr1   [10586, 11175]      - |         1
    [2]     chr1   [13149, 13328]      + |         1
    [3]     chr1   [13170, 13341]      + |         1
    [4]     chr1   [13304, 13494]      + |         1
    [5]     chr1   [13304, 13505]      + |         1
    ...      ...              ...    ... .       ...
  [217]     chr1 [115565, 115758]      - |         3
  [218]     chr1 [115650, 115757]      - |         1
  [219]     chr1 [115676, 115758]      - |         1
  [220]     chr1 [115650, 115758]      - |         1
  [221]     chr1 [115558, 115758]      - |         1
  -------
  seqinfo: 195 sequences from an unspecified genome

In [29]:
test_5000='/data/reddylab/Revathy/dev/Jamborees/MPRA_STARRseq/tmp/k562.atacstarr.output.rep3.dups_marked_subset5000.bam'
test_bam=BamFile(test_5000)

## yield.bam doesn't use isDuplicate=F flag while yield.bam1 uses the isDuplicate=F flag

test_out=reduceByYield(test_bam, yield.bam, map.bam, reduce.bam)
test_out_isdup=reduceByYield(test_bam, yield.bam1, map.bam, reduce.bam)

ctTable=collapse_reads(test_out)
ctTable_isdup=collapse_reads(test_out_isdup)

[1] 1793
[1] 895
[1] 952
[1] 895


In [33]:
head(ctTable, n=10)

GRanges object with 10 ranges and 1 metadata column:
       seqnames         ranges strand |     count
          <Rle>      <IRanges>  <Rle> | <integer>
   [1]     chr1 [10586, 11175]      - |         2
   [2]     chr1 [13149, 13328]      + |         1
   [3]     chr1 [13170, 13341]      + |         3
   [4]     chr1 [13304, 13494]      + |         3
   [5]     chr1 [13304, 13505]      + |         2
   [6]     chr1 [13304, 13505]      - |         2
   [7]     chr1 [13315, 13505]      - |         1
   [8]     chr1 [14453, 14620]      + |         1
   [9]     chr1 [14694, 15109]      + |         2
  [10]     chr1 [15180, 15401]      + |         3
  -------
  seqinfo: 195 sequences from an unspecified genome

In [32]:
head(ctTable_isdup, n=10)

GRanges object with 10 ranges and 1 metadata column:
       seqnames         ranges strand |     count
          <Rle>      <IRanges>  <Rle> | <integer>
   [1]     chr1 [10586, 11175]      - |         1
   [2]     chr1 [13149, 13328]      + |         1
   [3]     chr1 [13170, 13341]      + |         1
   [4]     chr1 [13304, 13494]      + |         1
   [5]     chr1 [13304, 13505]      + |         1
   [6]     chr1 [13304, 13505]      - |         2
   [7]     chr1 [13315, 13505]      - |         1
   [8]     chr1 [14453, 14620]      + |         1
   [9]     chr1 [14694, 15109]      + |         1
  [10]     chr1 [15180, 15401]      + |         2
  -------
  seqinfo: 195 sequences from an unspecified genome

In [52]:
input_500='/data/reddylab/Revathy/dev/Jamborees/MPRA_STARRseq/tmp/k562.atacstarr.input.rep1.dups_marked_subset500.bam'
input_bam=BamFile(input_500)

input_out=reduceByYield(input_bam, yield.bam, map.bam, reduce.bam)
input_out_isdup=reduceByYield(input_bam, yield.bam1, map.bam, reduce.bam)

input_ctTable=collapse_reads(input_out)
input_ctTable_isdup=collapse_reads(input_out_isdup)


Warning message in doTryCatch(return(expr), name, parentenv, handler):
“[bam_header_read] EOF marker is absent. The input is probably truncated.”Warning message in doTryCatch(return(expr), name, parentenv, handler):
“[bam_header_read] EOF marker is absent. The input is probably truncated.”Warning message in doTryCatch(return(expr), name, parentenv, handler):
“[bam_header_read] EOF marker is absent. The input is probably truncated.”Warning message in doTryCatch(return(expr), name, parentenv, handler):
“[bam_header_read] EOF marker is absent. The input is probably truncated.”

[1] 1756
[1] 1176
[1] 1071
[1] 1071


In [37]:
inp=BamFile('/data/reddylab/Revathy/dev/Jamborees/MPRA_STARRseq/tmp/k562.atacstarr.input.rep1.dups_marked_subset100.bam')
inp_out=reduceByYield(inp, yield.bam, map.bam, reduce.bam)
inp_out_isdup=reduceByYield(inp, yield.bam1, map.bam, reduce.bam)
inp_ct=collapse_reads(inp_out)
inp_ct_isdup=collapse_reads(inp_out_isdup)
inp_ct


[1] 45
[1] 44
[1] 44
[1] 44


GRanges object with 44 ranges and 1 metadata column:
       seqnames           ranges strand |     count
          <Rle>        <IRanges>  <Rle> | <integer>
   [1]     chr1   [10238, 10455]      - |         1
   [2]     chr1   [10586, 11024]      + |         1
   [3]     chr1   [14265, 14471]      + |         1
   [4]     chr1   [17336, 17499]      - |         2
   [5]     chr1   [17294, 17506]      - |         1
   ...      ...              ...    ... .       ...
  [40]     chr1 [115729, 115830]      + |         1
  [41]     chr1 [115729, 115799]      - |         1
  [42]     chr1 [115724, 115828]      - |         1
  [43]     chr1 [115729, 115835]      - |         1
  [44]     chr1 [115729, 115856]      - |         1
  -------
  seqinfo: 195 sequences from an unspecified genome

In [56]:
input_ctTable_isdup

GRanges object with 1071 ranges and 1 metadata column:
         seqnames           ranges strand |     count
            <Rle>        <IRanges>  <Rle> | <integer>
     [1]     chr1   [10238, 10455]      - |         1
     [2]     chr1   [10586, 11024]      + |         1
     [3]     chr1   [14265, 14471]      + |         1
     [4]     chr1   [17336, 17499]      - |         1
     [5]     chr1   [17294, 17506]      - |         1
     ...      ...              ...    ... .       ...
  [1067]     chr1 [634022, 634166]      - |         1
  [1068]     chr1 [634011, 634167]      - |         1
  [1069]     chr1 [634015, 634167]      - |         1
  [1070]     chr1 [634013, 634170]      - |         1
  [1071]     chr1 [634013, 634173]      - |         1
  -------
  seqinfo: 195 sequences from an unspecified genome

In [31]:
write.csv(ctTable, "/data/reddylab/Revathy/dev/Jamborees/MPRA_STARRseq/tmp/count_table.txt")
write.csv(ctTable_isdup, "/data/reddylab/Revathy/dev/Jamborees/MPRA_STARRseq/tmp/count_table_isDup.txt")

In [35]:
head(ctTable_isdup, n=10)

GRanges object with 10 ranges and 1 metadata column:
       seqnames         ranges strand |     count
          <Rle>      <IRanges>  <Rle> | <integer>
   [1]     chr1 [10586, 11175]      - |         1
   [2]     chr1 [13149, 13328]      + |         1
   [3]     chr1 [13170, 13341]      + |         1
   [4]     chr1 [13304, 13494]      + |         1
   [5]     chr1 [13304, 13505]      + |         1
   [6]     chr1 [13304, 13505]      - |         2
   [7]     chr1 [13315, 13505]      - |         1
   [8]     chr1 [14453, 14620]      + |         1
   [9]     chr1 [14694, 15109]      + |         1
  [10]     chr1 [15180, 15401]      + |         2
  -------
  seqinfo: 195 sequences from an unspecified genome

In [26]:
head(ctTable_isdup, n=10)

GRanges object with 10 ranges and 1 metadata column:
       seqnames         ranges strand |     count
          <Rle>      <IRanges>  <Rle> | <integer>
   [1]     chr1 [10586, 11175]      - |         1
   [2]     chr1 [13149, 13328]      + |         1
   [3]     chr1 [13170, 13341]      + |         1
   [4]     chr1 [13304, 13494]      + |         1
   [5]     chr1 [13304, 13505]      + |         1
   [6]     chr1 [13304, 13505]      - |         2
   [7]     chr1 [13315, 13505]      - |         1
   [8]     chr1 [14453, 14620]      + |         1
   [9]     chr1 [14694, 15109]      + |         1
  [10]     chr1 [15180, 15401]      + |         2
  -------
  seqinfo: 195 sequences from an unspecified genome

In [6]:
count_reads = function(reads) {
    print(length(reads))
    uniq = unique(reads)
    # sum over duplicates to get a count for each unique 5'/3' end
    reads$count = countOverlaps( reads, uniq, type="equal");
    print(length(uniq))
    return( reads );
}

count_reads(readFile)

[1] 29556329
[1] 29556329


GRanges object with 29556329 ranges and 1 metadata column:
                     seqnames           ranges strand |     count
                        <Rle>        <IRanges>  <Rle> | <integer>
         [1]             chr1   [10238, 10455]      - |         1
         [2]             chr1   [10586, 11024]      + |         1
         [3]             chr1   [14265, 14471]      + |         1
         [4]             chr1   [17336, 17499]      - |         1
         [5]             chr1   [17294, 17506]      - |         1
         ...              ...              ...    ... .       ...
  [29556325] chrUn_GL000218v1 [139140, 139236]      + |         1
  [29556326] chrUn_GL000218v1 [142191, 142579]      + |         1
  [29556327] chrUn_GL000218v1 [147527, 147670]      + |         1
  [29556328] chrUn_GL000218v1 [149210, 149539]      + |         1
  [29556329] chrUn_GL000218v1 [156032, 156188]      + |         1
  -------
  seqinfo: 195 sequences from an unspecified genome

In [ ]:
count_reads(out_read1)

In [ ]:
count_reads(out_read2)

In [41]:


# testfile='/data/reddylab/Revathy/dev/Jamborees/MPRA_STARRseq/tmp/k562.atacstarr.input.rep1.dups_marked_dedup.bam'
# input = BamFile(testfile, yieldSize=1 * 10^6, asMates=T)
# readFile = reduceByYield(input, yield.bam, map.bam, reduce.bam)

# out_test1='/data/reddylab/Alex/encode4_duke/data/data_distribution/bam/k562.atacstarr.output.rep3.dups_marked.bam'
# out_test2='/data/reddylab/Revathy/dev/Jamborees/MPRA_STARRseq/tmp/k562.atacstarr.output.rep3.dups_marked_dedup.bam'

# out1=BamFile(out_test1, yieldSize=1 * 10^6, asMates=T)
# out2=BamFile(out_test2, yieldSize=1 * 10^6, asMates=T)

# out_read1=reduceByYield(out1, yield.bam, map.bam, reduce.bam)
# out_read2=reduceByYield(out2, yield.bam, map.bam, reduce.bam)

test_dedup='/data/reddylab/Revathy/dev/Jamborees/MPRA_STARRseq/tmp/k562.atacstarr.output.rep3.dups_marked_subset500_dedup.bam'
test_dedup_bam=BamFile(test_dedup)
test_dedup_out=reduceByYield(test_dedup_bam, yield.bam, map.bam, reduce.bam)
test_dedup_out_isdup=reduceByYield(test_dedup_bam, yield.bam1, map.bam, reduce.bam)
ct_dedup=collapse_reads(test_dedup_out)
ct_dedup_isdup=collapse_reads(test_dedup_out_isdup)

[1] 128
[1] 124
[1] 128
[1] 124
